In [17]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import os
import pickle
import boto3

### holy shit
I am writing this on my couch tunnelling through ssh to an EC2 instance, this is the fucking future

In [18]:
iurl = "https://itch.io/games"
turl = "https://itch.io/tags"

In [19]:
stb = boto3.resource('s3')
for bucket in stb.buckets.all():
    print(bucket.name)

aws-logs-279455525701-us-west-1
derpbot
magonote-html
mascon
pxnx-test
yoon-galvanize-s3
yoon-test-bucket-00


In [20]:
test_data = "urggg! urk urk"
stb.Bucket('pxnx-test').put_object(Key='urg.txt', Body=test_data)

s3.Object(bucket_name='pxnx-test', key='urg.txt')

In [21]:
!echo "blaaaaargh! urg urg" > blerg.urg

# Create an S3 client
s3 = boto3.client('s3')

filename = 'blerg.urg'
bucket_name = 'pxnx-test'

# Uploads the given file using a managed uploader, which will split up large
# files automatically and upload parts in parallel.
s3.upload_file(filename, bucket_name, filename)

In [22]:
!aws s3 ls s3://pxnx-test

                           PRE scans/
                           PRE still/
2018-04-25 03:47:12         20 blerg.urg
2018-04-25 03:47:11         14 urg.txt


In [23]:
def gamelist(url):

    r = requests.get(url)
    rt = r.text
    # print(rt)
    soup = bs(rt, "lxml")
    lclass = "title game_link"
    allanc = soup.find_all("a", class_=lclass)
    # print(allanc)
    lset = set()
    for anc in allanc:
        lset.add(anc.get_text())
    return list(lset)

def taglist(url):
    # note this is for "top tags", not sure what the cutoff is
    r = requests.get(url)
    rt = r.text
    # print(rt)
    soup = bs(rt, "lxml")
    tclass = "outline_button"
    allanc = soup.find_all("a", class_=tclass)
    # print(allanc)
    tset = set()
    for anc in allanc:
        txt = anc.get_text()
        href = anc['href']
        lab = anc['data-label']
        ttup = (txt, href, lab)
        tset.add(ttup)
    return list(tset)

def tagscrape(taglist):
    tbase = "https://itch.io"
    tclist = list()
    for tag in taglist:
        turl = tbase + tag[1]
        r = requests.get(turl)
        rt = r.text
        # print(rt)
        soup = bs(rt, "lxml")
        cclass = "game_count"
        gcount = soup.find("nobr", class_=cclass)
        gctxt = gcount.get_text()
        gcdig = u''.join(c for c in gctxt if '0' <= c <= '9')
        gctup = (tag[0], tag[1], gcdig, tag[2])
        tclist.append(gctup)
    return tclist

def gamelist_from_file(file):
    with open (file, 'r') as fh:
        rt = fh.read()
    soup = bs(rt, "lxml")
    lclass = "title game_link"
    allanc = soup.find_all("a", class_=lclass)
    # print(allanc)
    lset = set()
    for anc in allanc:
        txt = anc.get_text()
        href = anc['href']
        lab = anc['data-label']
        gid = lab.split(':')
        ltup = (txt, href, int(gid[1]))
        lset.add(ltup)
    return list(lset)

In [26]:
# example url: <a class="title game_link" href="https://owch.itch.io/btab" data-label="game:231714:title" data-action="game_grid">BEAT THE ART BREAKER [DEMO]</a>

itchtags = taglist(turl)
# itchlist = gamelist(iurl)

# https://itch.io:443/games/tag-horrible
# https://itch.io:443/games/tag-horrible?page=2


In [27]:
tslist = tagscrape(itchtags[0:25])

In [28]:
tslist

[('Parody', '/games/tag-parody', '466', 'parody'),
 ('Western', '/games/tag-western', '215', 'western'),
 ('Mystery', '/games/tag-mystery', '1133', 'mystery'),
 ('nature', '/games/tag-nature', '260', 'nature'),
 ('FMV', '/games/tag-fmv', '31', 'fmv'),
 ('Massively multiplayer',
  '/games/tag-massively-multiplayer',
  '113',
  'massively-multiplayer'),
 ('Ludum Dare 30', '/games/tag-ludum-dare-30', '187', 'ludum-dare-30'),
 ('clicker', '/games/tag-clicker', '596', 'clicker'),
 ('Perma Death', '/games/tag-perma-death', '274', 'perma-death'),
 ('Horror', '/games/tag-horror', '2857', 'horror'),
 ('Photorealistic', '/games/tag-photorealistic', '77', 'photorealistic'),
 ('Visualization', '/games/tag-visualization', '64', 'visualization'),
 ('coming-of-age', '/games/tag-coming-of-age', '124', 'coming-of-age'),
 ('Top Down Adventure',
  '/games/tag-top-down-adventure',
  '438',
  'top-down-adventure'),
 ('Crafting', '/games/tag-crafting', '307', 'crafting'),
 ('Strategy', '/games/genre-strateg

In [14]:
# get my curl on
# looks like games end around page 3466 on 2018-04-23
# then 3515 on 2018-04-24
# most pages are around 65000 bytes(?)
# drops off to 26000 when there are no more games

istart = 1
# iend = 2345
ichunk = 173
irep = 2
izzz = 120

proj_reps = (3600 - istart) / ichunk
print(proj_reps)

20.803468208092486


In [13]:
# go get all that stuff

for rep in range(irep):
    
    rstart = (rep * ichunk) + istart
    rend = (((rep + 1) * ichunk) + istart) - 1
    srstart = str(rstart)
    srend = str(rend)
    prep = "rsrart: {} rend: {}".format(srstart, srend)
    print(prep)
    curl = "https://itch.io:443/games?page=[{}-{}]".format(srstart, srend)
    cfile = "html/big_itch_{}_{}.html".format(srstart, srend)

    !curl $curl >> $cfile

rsrart: 3500 rend: 3524

[1/25]: https://itch.io:443/games?page=3500 --> <stdout>
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 62558    0 62558    0     0   212k      0 --:--:-- --:--:-- --:--:--  212k

[2/25]: https://itch.io:443/games?page=3501 --> <stdout>
100 64593    0 64593    0     0   239k      0 --:--:-- --:--:-- --:--:-- 15.4M

[3/25]: https://itch.io:443/games?page=3502 --> <stdout>
100 63469    0 63469    0     0   233k      0 --:--:-- --:--:-- --:--:--  233k

[4/25]: https://itch.io:443/games?page=3503 --> <stdout>
100 62237    0 62237    0     0   195k      0 --:--:-- --:--:-- --:--:--  195k

[5/25]: https://itch.io:443/games?page=3504 --> <stdout>
100 62258    0 62258    0     0   235k      0 --:--:-- --:--:-- --:--:-- 14.8M

[6/25]: https://itch.io:443/games?page=3505 --> <stdout>
100 63965    0 63965    0     0   236k      0 --:--:-- --:--:-- --:--:--  2

In [16]:
htmdir = "html"

htmlist = list()
for file in os.listdir(htmdir):
    if file[-5:] == '.html':
        htmlist.append(file)

bilist = list()
for itchfile in htmlist:
    fpath = "html/" + itchfile
    sublist = gamelist_from_file(fpath)
    bilist.extend(sublist)
    
len(bilist)

NameError: name 'gamelist_from_file' is not defined

In [15]:
with open('all_games.pkl','wb') as fh:
    pickle.dump(bilist,fh)

NameError: name 'bilist' is not defined

In [16]:
for item in bilist[17:59]:
    curl = item[1]
    cfile = "html/game_pages/{}.html".format(item[2])

    !curl $curl > $cfile

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16108    0 16108    0     0   302k      0 --:--:-- --:--:-- --:--:--  302k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16266    0 16266    0     0   317k      0 --:--:-- --:--:-- --:--:--  317k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13785    0 13785    0     0   292k      0 --:--:-- --:--:-- --:--:--  292k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19374    0 19374    0     0   356k      0 --:--:-- --:--:-- --:--:--  363k
  % Total    % Received % Xferd  Average Speed   Tim

100 14145    0 14145    0     0   212k      0 --:--:-- --:--:-- --:--:--  212k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18749    0 18749    0     0   339k      0 --:--:-- --:--:-- --:--:--  339k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19359    0 19359    0     0   309k      0 --:--:-- --:--:-- --:--:--  309k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14674    0 14674    0     0   251k      0 --:--:-- --:--:-- --:--:--  251k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15385    0 15385    0     0   288k      0 --:--:

In [17]:
columns = ['name', 'url', 'gid']

df = pd.DataFrame([g for g in bilist], columns=columns)

# print df

df = df.set_index('gid')
#if you need set column name
df.columns.name = 'game_id'

df.sort_index()

game_id,name,url
gid,,
3,X-Moon,https://leafo.itch.io/x-moon
16,Escape From Andromeda,https://adit.itch.io/escape-from-andromeda
24,DARKLORD - An Arcade Platformer,https://tyoung.itch.io/darklord---an-arcade-pl...
28,Homunculus,https://doom-crow.itch.io/homunculus
37,PEA,https://gameofmax.itch.io/pea
39,Let's Learn Japanese: Deluxe,https://gzstorm.itch.io/lets-learn-japanese-de...
41,Adlib,https://spottedzebra.itch.io/adlib
42,Planet's Core,https://2bitent.itch.io/planetscore
44,Queen Frances and the Rat Rebellion,https://manikin.itch.io/rat-rebellion


In [18]:
df['url']

gid
176389                  https://egn-games.itch.io/run-block
240350                     https://mavw.itch.io/aeroproject
194540              https://link990.itch.io/orb-of-the-cave
188791    https://stentann.itch.io/phoenixgaming2dplatfo...
16032              https://letmethink.itch.io/slimeys-quest
32379             https://herocraft.itch.io/catch-the-candy
113493                  https://strdr4605.itch.io/wave-hero
193144                  https://ury18.itch.io/shark-rampage
163476      https://viashimo.itch.io/ld39-dead-in-the-water
14188                         https://raon.itch.io/the-cave
203393                 https://bobbybubi.itch.io/prep-story
199540    https://geneticastudios.itch.io/eljardindelafe...
225133                https://saagamestudio.itch.io/zomboid
163117     https://ministerjay.itch.io/wobbling-in-the-dark
164755                    https://radiosoap.itch.io/nothing
218325                  https://kandersen.itch.io/boundless
113576            https://becklespin

In [17]:
biglist[12]

('Kingdom Hearts nobody survives. (3d)',
 'https://horrornerd.itch.io/kingdom-hearts-nobody-survives',
 73957)